In [9]:
from pyensembl import EnsemblRelease

genome = EnsemblRelease(77)
genome.download()
genome.index()

INFO:pyensembl.download_cache:Fetching /home/ec2-user/.cache/pyensembl/GRCh38/ensembl77/Homo_sapiens.GRCh38.77.gtf.gz from URL https://ftp.ensembl.org/pub/release-77/gtf/homo_sapiens/Homo_sapiens.GRCh38.77.gtf.gz
INFO:datacache.download:Downloading https://ftp.ensembl.org/pub/release-77/gtf/homo_sapiens/Homo_sapiens.GRCh38.77.gtf.gz to /home/ec2-user/.cache/pyensembl/GRCh38/ensembl77/Homo_sapiens.GRCh38.77.gtf.gz
INFO:pyensembl.download_cache:Fetching /home/ec2-user/.cache/pyensembl/GRCh38/ensembl77/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL https://ftp.ensembl.org/pub/release-77/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
INFO:datacache.download:Downloading https://ftp.ensembl.org/pub/release-77/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz to /home/ec2-user/.cache/pyensembl/GRCh38/ensembl77/Homo_sapiens.GRCh38.cdna.all.fa.gz
INFO:pyensembl.download_cache:Fetching /home/ec2-user/.cache/pyensembl/GRCh38/ensembl77/Homo_sapiens.GRCh38.ncrna.fa.gz from URL htt

In [11]:
label = 'ENSG00000002079'

In [12]:
gene = genome.gene_by_id(label)

# Observed and predicted gene expression values were obtained by summing up the observed/predicted CAGE read counts
# at all unique TSS locations of the gene. For each TSS location, we used the 128-bp bin overlapping the TSS as well
# as the two neighboring bins (3 bins in total).

# For each gene, look through all transcipts - protein coding / not, and record offsets from gene start
gene.transcripts

[Transcript(transcript_id='ENST00000439784', transcript_name='MYH16-001', gene_id='ENSG00000002079', biotype='unitary_pseudogene', contig='7', start=99238794, end=99311130, strand='+', genome='GRCh38'),
 Transcript(transcript_id='ENST00000453194', transcript_name='MYH16-002', gene_id='ENSG00000002079', biotype='processed_transcript', contig='7', start=99238911, end=99258156, strand='+', genome='GRCh38'),
 Transcript(transcript_id='ENST00000413734', transcript_name='MYH16-003', gene_id='ENSG00000002079', biotype='processed_transcript', contig='7', start=99258210, end=99261636, strand='+', genome='GRCh38'),
 Transcript(transcript_id='ENST00000425880', transcript_name='MYH16-004', gene_id='ENSG00000002079', biotype='processed_transcript', contig='7', start=99264267, end=99271049, strand='+', genome='GRCh38'),
 Transcript(transcript_id='ENST00000452010', transcript_name='MYH16-005', gene_id='ENSG00000002079', biotype='processed_transcript', contig='7', start=99297912, end=99301804, strand=

In [13]:
len(genome.protein_ids())

99436

In [14]:
protein_id = genome.protein_ids()[0]
genome.protein_sequence(protein_id)

# results = []
# for protein_id in genome.protein_ids():
#     sequence = genome.protein_sequence(protein_id)
#     if '*' not in sequence:
#         results.append({
#             'protein_id': protein_id,
#             'protein_sequence': sequence
#         })

'MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTIPTIGFNVETVEYKNICFTVWDVGGQDKIRPLWRHYFQNTQGLIFVVDSNDRERVQESADELQKMLQEDELRDAVLLVFANKQDMPNAMPVSELTDKLGLQHLRSRTWYVQATCATQGTGLYDGLDWLSHELSKR'